# Using Decision Tree & Naive Bayes to classify Legendary Pokemon

## Import libraries

In [1]:
import gc
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import pandas_profiling as pdp
# from sklearn.feature_extraction import FeatureHasher
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
import utils as ut
import decisionTree as dt
import naiveBayes as nb

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [ ]:
# Set packages options
# np.set_printoptions(threshold=np.nan)
pd.set_option("display.max_columns", 600)
plt.rcParams["figure.figsize"] = (11, 6)

## Define constants and functions

In [3]:
# Constants
TypeColorMappings = {"Water": "#6890F0", "Fire": "#F08030", "Grass": "#78C850",
                     "Dark": "#705848", "Electric": "#F8D030", "Flying": "#A890F0",
                     "Normal": "#A8A878", "Fighting": "#C03028", "Poison": "#A040A0",
                     "Ground": "#E0C068", "Psychic": "#F85888", "Rock": "#B8A038", 
                     "Ice": "#98D8D8", "Bug": "#A8B820", "Dragon": "#7038F8", 
                     "Ghost": "#705898", "Steel": "#B8B8D0", "Fairy": "#EE99AC"}

# DropColumns = ["Pokedex#", "Name", "Type 1", "Type 2", "Generation", "Ability 1", "Ability 2", "Ability 3", 
#                "EggGroup 1", "EggGroup 2", "Category", "Height (m)", "Weight (kg)"]

DropColumns = ["Pokedex#", "Name", "Generation", "Category"]

def getColorList(typeCounts):
    assert type(typeCounts) == pd.core.series.Series, "Argument must be a Series object"
    return [TypeColorMappings[pokemonType] for pokemonType in typeCounts.index]

def getDistinctValues(dataFrame, columnName, sep):
    result = []
    for value in dataFrame[columnName].value_counts().index:
        result += value.split(sep)
    return set(result)

## Load the data from file

In [4]:
# Load the Pokemon data
fileName = r'.\Pokemon_Cleaned.tsv'
columnTypes = {"Name": str, "Category": str, "Type 1": str, "Type 2": str, 
               "Ability 1": str, "Ability 2": str, "Ability 3": str, "Group": str}
data = pd.read_csv(fileName, header=0, sep='\t', dtype=columnTypes)
data.head()

,Generation,Pokedex#,Name,Category,Type 1,Type 2,Ability 1,Ability 2,Ability 3,MaleRatio,...,EggGroup 1,EggGroup 2,HP,Attack,Defense,Sp.Attack,Sp.Defense,Speed,Total,Group
0,1.0,1.0,Bulbasaur,Seed Pokemon,Grass,Poison,Chlorophyll,Overgrow,None,87.5,...,Monster,Grass,45.0,49.0,49.0,65.0,65.0,45.0,318,Ordinary
1,1.0,2.0,Ivysaur,Seed Pokemon,Grass,Poison,Chlorophyll,Overgrow,None,87.5,...,Monster,Grass,60.0,62.0,63.0,80.0,80.0,60.0,405,Ordinary
2,6.0,3.0,Mega Venusaur,Seed Pokemon,Grass,Poison,Thick Fat,None,None,87.5,...,Monster,Grass,80.0,100.0,123.0,122.0,120.0,80.0,625,Ordinary
3,1.0,3.0,Venusaur,Seed Pokemon,Grass,Poison,Chlorophyll,Overgrow,None,87.5,...,Monster,Grass,80.0,82.0,83.0,100.0,100.0,80.0,525,Ordinary
4,1.0,4.0,Charmander,Lizard Pokemon,Fire,None,Blaze,Solar Power,None,87.5,...,Monster,Dragon,39.0,52.0,43.0,60.0,50.0,65.0,309,Ordinary


## Plot graphs to visualize and understand the data

In [ ]:
# Plot Group occurences to see the distributions
groupFig, groupAxes = plt.subplots(nrows=1, ncols=1)
groupCounts = data["Group"].value_counts()
groupAxe = groupCounts.plot(title="Group", kind="bar")
groupAxe.set(xlabel="Group Types", ylabel="Count")

In [ ]:
# Box plot Total & Group
data.boxplot("Total", "Group")

<p> According to the boxplot above, we can see that despite having a small quanity in size, Legendary Pokemon has the highest Total stats compared to the other 2 groups.</p>

## PCA

In [ ]:
reducedData = data.drop(DropColumns, axis=1)
# dummies = pd.get_dummies(data[["Type 1", "Type 2", "Ability 1", "Ability 2", "Ability 3", "EggGroup 1", "EggGroup 2"]])
# dataWithDummies = pd.concat([dummies, reducedData], axis=1, join_axes=[reducedData.index])

# x = dataWithDummies.loc[:, dataWithDummies.columns != "Group"]
# y = dataWithDummies.loc[:, "Group"]

# Scale the x data
#x = StandardScaler().fit_transform(x)

In [ ]:
pca = PCA(0.95)
principalComponents = pca.fit_transform(x)
width, height = principalComponents.shape
principalDf = pd.DataFrame(data=principalComponents, columns=["pca %d" % i for i in range(1, height + 1)])
pcaData = pd.concat([principalDf, dataWithDummies[["Group"]]], axis=1)

xTrain = pcaData.sample(frac=0.5)

In [ ]:
# # Plot Type 1 and Type 2 occurences to see the distributions
# typeFig, typeAxes = plt.subplots(nrows=3, ncols=1)
# typeFig.subplots_adjust(top=3)

# type1Counts = data["Type 1"].value_counts()
# type2Counts = data["Type 2"].value_counts().drop("None")
# typesCounts = type1Counts.add(type2Counts, fill_value=0)

# type1Counts.plot(title="Type 1 Occurrences", kind="bar", ax=typeAxes[0], color=getColorList(type1Counts))
# type2Counts.plot(title="Type 2 Occurrences", kind="bar", ax=typeAxes[1], color=getColorList(type2Counts))
# typesCounts.plot(title="Type 1 + 2 Occurrences", kind="bar", ax=typeAxes[2], color=getColorList(typesCounts))

# fh = FeatureHasher(n_features=2, input_type="string")
# hashedFeature1 = fh.fit_transform(data["Type 1"])
# hashedFeature2 = fh.fit_transform(data["Type 2"])

# x = pd.concat([data[["Name", "Type 1", "Type 2", "Category"]], pd.DataFrame(hashedFeature1.toarray())], axis=1)
# x = pd.concat([x, pd.DataFrame(hashedFeature2.toarray())], axis=1)

## Decision Tree

In [ ]:
reducedData = data.drop(DropColumns, axis=1)
training, test = ut.splitData(reducedData, 0.60)
dtree = dtree.DecisionTree("Group")
dtree.train(training)
pred = dtree.classify(test)

In [ ]:
matrix = ut.buildConfusionMatrix(pred, test[dtree.targetFeature], reducedData[dtree.targetFeature].unique())
precisions, recalls = ut.getPrecisionsAndRecalls(matrix, reducedData[dtree.targetFeature].unique())
fScores = ut.computeFScores(precisions, recalls)

In [ ]:
fScores

In [ ]:
gc.collect()
m = pd.DataFrame({"bool": [True, False, False], "number": [1,2,3]})
isinstance(m.dtypes["bool"].type, np.bool_)
m.dtypes["bool"].type == np.bool_

### Naive Bayes

In [ ]:
import naiveBayes as nb

In [5]:
reducedData = data.drop(DropColumns, axis=1)
training, test = ut.splitData(reducedData, 0.60)
# nBayes.train(training)

In [48]:
nBayes = nb.NaiveBayes("Group")
d = nBayes.train(training)
# nBayes._getFeatureType(data, "Name")

****Skipping MaleRatio feature for label Ultra Beast. Standard deviation = 0
****Skipping FemaleRatio feature for label Ultra Beast. Standard deviation = 0


In [50]:
nBayes._continuousProbTable

,MaleRatio=mean,MaleRatio=std,FemaleRatio=mean,FemaleRatio=std,Height (m)=mean,Height (m)=std,Weight (kg)=mean,Weight (kg)=std,HP=mean,HP=std,...,Defense=mean,Defense=std,Sp.Attack=mean,Sp.Attack=std,Sp.Defense=mean,Sp.Defense=std,Speed=mean,Speed=std,Total=mean,Total=std
Ordinary,50.603622,23.949982,43.762575,23.129696,1.042455,0.773521,49.750101,76.768623,67.209256,25.283005,...,70.291751,29.564879,67.804829,29.968467,69.573441,26.497092,64.965795,27.805431,415.301811,106.799620
Legendary,12.068966,31.498183,5.172414,20.282339,2.077586,1.352197,185.417241,222.266148,87.793103,21.986247,...,98.551724,28.846948,113.793103,31.827252,104.793103,28.480685,97.000000,25.574110,611.551724,65.736361
Ultra Beast,NaN,NaN,NaN,NaN,2.850000,3.024661,281.112500,406.232264,78.000000,22.703052,...,98.500000,59.605848,95.000000,40.313061,75.000000,34.706936,87.000000,40.792156,551.250000,53.033009


In [51]:
nBayes._categoricalProbTable

,Type 1=Normal,Type 1=Water,Type 1=Bug,Type 1=Grass,Type 1=Rock,Type 1=Fire,Type 1=Electric,Type 1=Dark,Type 1=Psychic,Type 1=Ghost,...,Ability 1=Psychic Surge,Ability 1=Desolate Land,Ability 1=Natural Cure,Ability 2=Aura Break,Ability 2=Misty Surge,Ability 2=Multiscale,Ability 2=Ice Body,Ability 2=Bulletproof,Ability 2=Grassy Surge,Ability 1=Beast Boost
Ordinary,0.140845,0.136821,0.100604,0.088531,0.068410,0.062374,0.050302,0.044266,0.042254,0.040241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Legendary,0.068966,0.068966,NaN,0.051724,0.068966,0.051724,0.086207,0.017241,0.241379,0.017241,...,0.017241,0.017241,0.017241,0.034483,0.017241,0.017241,0.017241,0.017241,0.017241,NaN
Ultra Beast,NaN,NaN,0.250000,0.125000,0.250000,0.125000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [43]:
training[training["Group"] == "Ordinary"]["Height (m)"].mean()

1.0424547283702215

In [ ]:
training[training["Group"] == "Ultra Beast"].describe()

In [ ]:
training.head()

In [ ]:
for i in training["FemaleRatio"]: print(i)

In [ ]:
mean = training["FemaleRatio"].mean()
math.sqrt(sum(map(lambda x: (x-mean)**2, training["FemaleRatio"])) / len(training))

In [ ]:
s = [1 for i in range(100)]
mean = sum(s) / 100
print(mean)
math.sqrt(sum(map(lambda x: (x-mean)**2, s)) / len(training))

In [18]:
value = 1.1
mean = 20
std = 5
prob = (1 / (math.sqrt(2 * (std**2) * math.pi))) * math.exp(-(value - mean)**2 / (2 * std**2))
prob

6.298736258150442e-05

In [ ]:
def filter(dataFrame, column, value):
    return dataFrame[dataFrame[column] == value]

In [ ]:
filter(training, "Type 1", "Normal")["Group"].value_counts()

In [ ]:
training["Group"].value_counts()

In [ ]:
training["Type 1"].value_counts()

### Plot Error Rate

In [ ]:
ratios = [0.2, 0.4, 0.6, 0.8]
dt = DecisionTree("Group")
x = []
y = []

for ratio in ratios:
    print("************ Split ratio: ", ratio)
    training, test = splitData(data, ratio)
    node = dt.train(training)
    predictions = dt.classify(test, node)
    error = computeError(predictions, test[dt.targetFeature])
    
    x.append(len(training))
    y.append(error)

In [ ]:
plt.plot(x, y, 'bo-', label="With categorical and continuous features")
plt.xlabel("Trainging Set size")
plt.ylabel("Error Rate")
plt.legend(loc="best")

In [ ]:
# Get data profile
profile = pdp.ProfileReport(dataWithDummies)
profile.to_file("Profile.html")
profile = None
gc.collect()